In [1]:
import bz2
import torch
from torch import nn
import numpy as np
import copy
import torch.optim as optim
import re


In [2]:
bz_file = bz2.BZ2File('D:/Installation/1305_800230_compressed_test.ft.txt.bz2/test.ft.txt.bz2')
file_name_model_latest_version = 'Model/model_latest_version.pt'
line_list = bz_file.readlines()
training_data_truncation = 2
training_data = line_list[0: training_data_truncation]

In [3]:
training_input_data = [None for i in range(0, len(training_data))]
training_output_data = [None for i in range(0, len(training_data))]

training_output_data = [str(x).split('__label__')[1][0] for x in training_data]
training_input_data = [re.sub('[^A-Za-z0-9]+'," ",str(str(x).split('__label__')[1][2: -4])) for x in training_data]
#training_input_data[1]
#training_output_data[1]


#how to keep "I'll" kind of word
#sort out words by frequency, and only keep the word with top 5000 frequency, and others to be inside 1 group
#how to group same meaning of words together?
#similiar meaning of words to be together
#transfer it into array format before pushing into final machine learning model
#make all as lower letter and remove duplication
#s,es etc.
#how to keep the scetence structure?
#in there a better way to save the matrix with most of the value is 0?
#remove word only appear once

In [4]:
def list_padding(original_list):
    max_len = len(max(original_list, key = len))
    new_list = []
    for m in range(0, len(original_list)):
        for i in range(0, max_len - len(original_list[m])):
            original_list[m].append(None)
    return original_list, max_len
def list_to_array_through_dict(original_list, original_dict):
    new_list, list_len = list_padding(original_list)
    dict_len = len(original_dict)
    array_template = [[0 for i in range(dict_len)] for j in range(list_len)]
    #print(array_template)
    final_list = []
    for item in new_list:
        new_list_item = copy.deepcopy(array_template)
        for j in range(len(item)):
            if item[j] != None:
                new_list_item[j][original_dict[item[j]]] = 1
        final_list.append(new_list_item)
    return final_list

def output_list_to_array(output_list):
    list_len = len(output_list)
    output_len = 2
    array_template = [0 for i in range(output_len)]
    #print(array_template)
    final_list = []
    for item in output_list:
        new_list_item = copy.deepcopy(array_template)
        new_list_item[int(item) - 1] = 1
        final_list.append(new_list_item)
    return final_list

In [5]:
words = set((' '.join(training_input_data)).split())
int2word = dict(enumerate(words))
vocab_size = len(int2word)

word2int = {word: ind for ind, word in int2word.items()}
word2int

training_input_data_split = [x.split() for x in training_input_data]

max_len = len(max(training_input_data_split, key = len))
#training_input_data_split
print(max_len)

#training_input_data_split = [for i in range(max_len - len(x)): x.insert(0, "") for x in training_input_data_split]
int2word
#new_list = list_padding(training_input_data_split)
#print(new_list)
print(len(int2word))


#final_input = torch.Tensor(list_to_array_through_dict(training_input_data_split, word2int))

#final_output = torch.Tensor(output_list_to_array(training_output_data))
#print(final_output)
final_input = np.array(list_to_array_through_dict(training_input_data_split, word2int))
final_output = np.array(output_list_to_array(training_output_data))
final_input = torch.Tensor(final_input)
final_output = torch.Tensor(final_output)

152
161


In [15]:
#len(new_list[0][0])
kkk = list_to_array_through_dict(training_input_data_split, word2int)
kkk = np.array(kkk)
print(kkk.shape)

mm = final_input.size()
print(mm)

final_input.size(0)

(2, 152, 161)
torch.Size([2, 152, 161])


2

In [7]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cpu


In [8]:
print(final_input)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


In [37]:
class LSTM_model(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(LSTM_model, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        print(batch_size)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        #out = self.sigmoid(out)
        
        #out = out.view(batch_size, -1)
        #print(out)
        #out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [38]:
#def training_model():
learning_rate = 0.3
epoch_size = 2
steps_for_printing_out_loss = 1

output_size = 2
#output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

LSTM_model_training_WIP = LSTM_model(vocab_size = vocab_size, output_size = output_size, embedding_dim = embedding_dim, hidden_dim = hidden_dim, n_layers = n_layers)
#RNN_model.to(device)
loss_functioin = nn.MSELoss()
optimizer = optim.SGD(LSTM_model_training_WIP.parameters(), lr = learning_rate)

for training_step_no in range(0, training_data_truncation):
    input = final_input[training_step_no]
    target = final_output[training_step_no]
    for i in range(1, epoch_size + 1):
        optimizer.zero_grad()
        output, hidden = LSTM_model_training_WIP(input)
        print(output)
        print(output.size())
        #print(hidden)
        #print(hidden.size)
        loss = loss_functioin(output, target.reshape(output.size(0), output.size(1)))
        loss.backward()
        if i % (steps_for_printing_out_loss) == 0:
            print('Loss (epoch: ' + str(i) + '): ' + str(loss.cpu().detach().numpy()))
        optimizer.step()

torch.save({'state_dict': LSTM_model_training_WIP.state_dict(),'optimizer': optimizer.state_dict()}, file_name_model_latest_version)

152
tensor([[-0.0150, -0.0197],
        [-0.0322,  0.0022],
        [-0.0289, -0.0433],
        ...,
        [-0.0423, -0.0400],
        [-0.0786,  0.0417],
        [ 0.0402, -0.0878]], grad_fn=<ThAddmmBackward>)
torch.Size([24472, 2])


RuntimeError: shape '[24472, 2]' is invalid for input of size 2